## Use PhariaSearch with PhariaKernel Skills

### Load the Enviroment Variables

In [ ]:
from os import getenv

from dotenv import load_dotenv

load_dotenv(override=True)

NAMESPACE = getenv("PHARIA_DATA_NAMESPACE")
COLLECTION = getenv("PHARIA_DATA_COLLECTION")
INDEX = getenv("INDEX")

### Define the Skill

PhariaSearch is natively integrated within PhariaKernel to reduce the effort necessary to develop a RAG based system.

The PhariaSkill SDK is available on PyPI (https://pypi.org/project/pharia-skill/) and can be installed with `pip install pharia-skill`. Let's start by importing the necessary dependencies.

<div class="alert alert-info">
PhariaKernel supports only semantic search and not filter indexes or hybrid indexes. This will be supported in the near future.
</div>

In [ ]:
from pharia_skill import ChatParams, Csi, IndexPath, Message, skill
from pydantic import BaseModel

Let's now define the basic Input and Output structure to make our RAG skill works.

In [ ]:
class Input(BaseModel):
    question: str
    namespace: str = NAMESPACE
    collection: str = COLLECTION
    index: str = INDEX


class Output(BaseModel):
    answer: str | None

Finally, let's define our AI logic within the skill.

In [ ]:
@skill
def rag_skill(csi: Csi, input: Input) -> Output:
    index = IndexPath(
        namespace=input.namespace,
        collection=input.collection,
        index=input.index,
    )

    if not (documents := csi.search(index, input.question, 3, 0.5)):
        return Output(answer=None)

    context = "\n".join([d.content for d in documents])
    content = f"""Using the provided context documents below, answer the following question accurately and comprehensively. If the information is directly available in the context documents, cite it clearly. If not, use your knowledge to fill in the gaps while ensuring that the response is consistent with the given information. Do not fabricate facts or make assumptions beyond what the context or your knowledge base provides. Ensure that the response is structured, concise, and tailored to the specific question being asked.

Input: {context}

Question: {input.question}
"""
    message = Message.user(content)
    params = ChatParams(max_tokens=512)
    response = csi.chat("llama-3.1-8b-instruct", [message], params)
    return Output(answer=response.message.content)

### Test your skill with DevCSI

Now that the Skill is defined, it is possible to test it locally using the DevCSI provided by Kernel, that requires two extra environment variables:

```bash
PHARIA_AI_TOKEN=
PHARIA_KERNEL_ADDRESS=
```

In [ ]:
from pharia_skill.testing import DevCsi

csi = DevCsi()
result = rag_skill(csi, Input(question="What is the influence of Robert Moses?"))

print(result.answer)

### Move the skill to production

Once the Skill is ready, it is possible to deploy it to production by following the steps in the <a href="https://docs.aleph-alpha.com/products/pharia-ai/pharia-studio/pharia-kernel/quick_start">Quick Start with Kernel skills</a>